In [ ]:
import pandas as pd
import numpy as np
import sys
%autoreload 2
%matplotlib inline
sys.path.append('..')

# Transactions -> Domains

Read data and convert to domain sets for each `machine_id, week` pair

In [ ]:
panels = pd.read_csv("../output/all_panels.csv", index_col=0)

transactions_fpath = '../data/comscore/2017/transactions.csv'
transactions_df = pd.read_csv(transactions_fpath, parse_dates=['event_date'])[['machine_id', 'event_date', 'domain_name']]

In [ ]:
tdf = (transactions_df
 .assign(week=lambda x: x.event_date.dt.week)
 .dropna()
 .groupby(['machine_id', 'week'])
 .agg({'domain_name': set}).reset_index()
       .assign(n_domains=lambda x: x.domain_name.map(len))
       .assign(domain_name=lambda x: x.domain_name.map(lambda y: "|".join(list(y))))
      )

In [ ]:
tdf.sort_values('n_domains', ascending=False).head(10)

In [ ]:
tdf.to_csv('../output/domains_by_week.csv')

## Fake Floc

1. assign domain list history to machine IDs
2. For each panel, calculate cohort assignment for a given week
3. calculate t-closeness across stratified variables and other variables of interest

In [ ]:
from floc import simulate

In [ ]:
simulate(tdf.iloc[0].domain_name.split("|"))

In [ ]:
import itertools
unique_domains = list(set(itertools.chain(*[d.split("|") for d in tdf.domain_name])))

def pad_domains(l, s):
    n = 7 - len(l)
    if n > 0:
        return list(l) + list(np.random.choice(s, n))
    else:
        return l

tdf['padded_domain'] = tdf.apply(lambda x: pad_domains(x.domain_name.split("|"), unique_domains), axis=1)

In [ ]:
len(unique_domains)

In [ ]:
import requests
etlds = [requests.get("http://" + d).url for d in unique_domains]

Find all "blocked" domains from FloC 

In [ ]:
blocked_domains = list(set(itertools.chain(*blocked)))

In [ ]:
ds = [[d] * 7 for d in unique_domains]
blocked_domains = []
for d in ds:
    try:
        simulate(d)
    except Exception as e:
        print(e)
        blocked_domains.append(d[0])
    

In [ ]:
blocked_domains

In [ ]:
import socket

In [ ]:
socket.getfqdn('ww.' + blocked_domains[0])

In [ ]:
padded_domains = [d for d in tdf.padded_domain]

In [ ]:
cohorts = []
blocked = []
for d in padded_domains:
    try:
        cohorts.append((d, simulate(d)))
    except Exception as e:
        blocked.append(d)

In [ ]:
len(cohorts)

In [ ]:
len(blocked)

In [ ]:
tdf.padded_domain[3]

In [ ]:
sorting_cluster_data = ""

In [ ]:
tdf['cohort'] = [simulate(domains, sorting_cluster_data) for domains in tdf.padded_domain]